<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Finance with Python

**_A Gentle Introduction_**

### Contingent Claim Valuation &mdash; Incomplete Markets

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Option Valuation

Different techniques are considered:

* discounting (wrong way)
* replication (linear algebra)
* volatility (standard deviation)
* approximation (OLS regression)
* neural nets (learning algorithm)
* martingale pricing (probabilistic approach)
* superreplication (safety approach)
* discounting (best try)

## Simple Financial Market

**_Two dates only (today, tomorrow), two traded assets and THREE future states only with equal probability._**

### Traded Financial Assets

In [ ]:
!git clone https://github.com/tpq-classes/finance_with_python.git
import sys
sys.path.append('finance_with_python')


In [ ]:
import numpy as np
np.set_printoptions(suppress=True)

In [ ]:
# risky stock -- future payoff
S = np.array((20, 10, 5))

In [ ]:
# risky stock -- current price
S0 = 10

In [ ]:
# risk-less bond -- future payoff
B = np.array((11, 11, 11))

In [ ]:
# risk-less bond -- current price
B0 = 10

### The Market 

In [ ]:
# market price vector
M0 = np.array((S0, B0))

In [ ]:
# market payoff matrix
M = np.array((S, B)).T
M

### A European Call Option 

In [ ]:
# contingent claim -- future payoff
K = 14.5
C = np.maximum(S - K, 0)
C

## Expected Returns

In [ ]:
def ret(x0, x):
    return (x / x0 - 1).mean()

In [ ]:
rS = ret(S0, S)
rS

In [ ]:
rB = ret(B0, B)
rB

## Discounting &mdash; Wrong Way

In [ ]:
# stock expected return
C.mean() / (1 + rS)

In [ ]:
# bond expected return
C.mean() / (1 + rB)

In [ ]:
# stock expected return, given risk-free return
S.mean() / (1 + rB)

## Replication &mdash; Linear Algebra

In [ ]:
# solving system of linear equations -- not possible here
# phi = np.linalg.solve(M, C)

## Volatility &mdash; Standard deviation

In [ ]:
# volatility ...
def vol(x0, x):
    return (x / x0 - 1).std()

In [ ]:
# ... of stock
vol(S0, S)

In [ ]:
# ... of bond
vol(B0, B)

## Approximation &mdash; OLS Regression

In [ ]:
phi = np.linalg.lstsq(M, C, rcond=None)[0]
phi

In [ ]:
np.dot(M, phi)

In [ ]:
np.dot(M, phi) - C

## Neural Nets &mdash; Learning Algorithm

<img src="http://hilpisch.com/images/neural_net.png">

In [ ]:
# intial weights
w = np.random.standard_normal(2)
w

In [ ]:
# layer one (output)
l1 = np.dot(M, w)
l1

In [ ]:
# deltas
d = l1 - C
d

In [ ]:
# mean squared error
(d ** 2).mean()

In [ ]:
# learning rate
alpha = 0.001

In [ ]:
# update values, via the dot product with M transposed
u = alpha * np.dot(M.T, d)
u

In [ ]:
# updating weights -- this approach is now possible again
w -= u
w

In [ ]:
# running a loop
w = np.random.standard_normal(2)
for i in range(100):
    l1 = np.dot(M, w)
    d = l1 - C
    u = alpha * np.dot(M.T, d)
    w -= u
    if i % 10 == 0:
        print((d ** 2).mean())

In [ ]:
l1 = np.dot(M, w)
l1

In [ ]:
np.dot(M, phi)

## Martingale Pricing &mdash; Probabilistic Approach

In [ ]:
# making the stock price a martingale
def E(P):
    return np.dot(S, P) / (1 + rB)

In [ ]:
P = np.array((0.3, 0.2, 0.5))
P.sum()

In [ ]:
E(P)

In [ ]:
E(P) == S0

In [ ]:
from scipy.optimize import minimize

In [ ]:
def error(P):
    return (E(P) - S0) ** 2

In [ ]:
error(P)

In [ ]:
bnds = ((0, 1), (0, 1), (0, 1))

In [ ]:
cons = {'type': 'eq', 'fun': lambda Q: Q.sum() - 1}

In [ ]:
P_ = 3 * [1 / 3]

In [ ]:
opt = minimize(error, P, bounds=bnds, constraints=cons)  # replace P by e.g. P_ or P * 2

In [ ]:
opt

In [ ]:
round(opt['fun'], 7)

In [ ]:
Q = opt['x']
Q

In [ ]:
E(Q)

In [ ]:
round(E(Q), 6) == S0

In [ ]:
error(Q)

In [ ]:
S.mean() / (1 + rB)  # wrong discounting (with equal probability measure)

In [ ]:
np.dot(S, Q) / (1 + rB)  # correct discounting (with martingale measure)

In [ ]:
C0_rn = np.dot(C, Q) / (1 + rB)  # correct discounting (with martingale measure)
C0_rn

## Superreplication &mdash; Safety Approach

In [ ]:
# naive approach (using the bond only)
phi = np.max(C) / B[0]
phi

In [ ]:
phi * B

In [ ]:
phi * B >= C

In [ ]:
phi * B - C

In [ ]:
phi * B0  # cost of bond portfolio

In [ ]:
# naive approach (using the stock and bond)
phi = np.array((0.45, -0.2))
np.dot(M, phi)

In [ ]:
np.dot(M, phi) - C

In [ ]:
np.dot(M0, phi)  # cost of stock+bond portfolio

In [ ]:
# cost-minimizing approach

In [ ]:
def cost(phi):
    return np.dot(M0, phi)

In [ ]:
cost(phi)

In [ ]:
cons = {'type': 'ineq', 'fun': lambda phi: np.dot(M, phi) - C}

In [ ]:
opt = minimize(cost, phi, constraints=cons)
opt

In [ ]:
C0_sr = opt['fun']  # present value if superreplication is the criterion
C0_sr

In [ ]:
phi = opt['x']
phi

In [ ]:
np.dot(M, phi)

In [ ]:
np.dot(M, phi).round(7) >= C

## Discounting &mdash; Best Try

In [ ]:
# expected return of option
rC_rn = ret(C0_rn, C)
rC_rn

In [ ]:
# expected return of option
rC_sr = ret(C0_sr, C)
rC_sr

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>